In [126]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import random

def load_winning_numbers(file_path):
    df = pd.read_excel(file_path, header=None)
    return df.values.tolist()

def calculate_transition_probabilities(numbers):
    transitions = defaultdict(lambda: defaultdict(int))
    for i in range(len(numbers) - 1):
        current = tuple(sorted(numbers[i]))
        next_num = tuple(sorted(numbers[i+1]))
        transitions[current][next_num] += 1
    
    for current in transitions:
        total = sum(transitions[current].values())
        for next_num in transitions[current]:
            transitions[current][next_num] /= total
    
    return transitions

def predict_next_numbers(last_numbers, transitions, num_predictions=5):
    current = tuple(sorted(last_numbers))
    if current not in transitions:
        print("Warning: 이전에 나온 적 없는 조합입니다. 랜덤 예측을 수행합니다.")
        return [(sorted(random.sample(range(1, 46), 6)), 1/8145060) for _ in range(num_predictions)]
    
    probabilities = transitions[current]
    predictions = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)[:num_predictions]
    return [(list(pred[0]), pred[1]) for pred in predictions]

def analyze_number_frequency(numbers):
    all_numbers = [num for combo in numbers for num in combo]
    frequency = Counter(all_numbers)
    total_draws = len(numbers)
    return {num: count / total_draws for num, count in frequency.items()}

def calculate_expected_value(number_frequency):
    probabilities = list(number_frequency.values())
    return np.mean(probabilities) * 6  # 6개

def analyze_top_numbers_winning(numbers, top_n=10):
    number_frequency = analyze_number_frequency(numbers)
    top_numbers = sorted(number_frequency.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_numbers_set = set(num for num, _ in top_numbers)
    
    matches = []
    for combo in numbers:
        match_count = len(set(combo) & top_numbers_set)
        matches.append(match_count)
    
    match_distribution = Counter(matches)
    total_draws = len(numbers)
    
    print(f"\n상위 {top_n}개 번호 중 당첨된 번호 개수 분포:")
    for match_count, count in sorted(match_distribution.items()):
        percentage = (count / total_draws) * 100
        print(f"{match_count}개 일치: {count}회 ({percentage:.2f}%)")
    
    average_matches = sum(match * count for match, count in match_distribution.items()) / total_draws
    print(f"\n평균 일치 개수: {average_matches:.2f}")

    return top_numbers, match_distribution, average_matches

def predict_with_top_numbers(last_numbers, top_numbers, num_predictions=5):
    top_numbers_set = set(num for num, _ in top_numbers)
    last_numbers_set = set(last_numbers)
    
    # 상위 번호 중 마지막 당첨 번호에 없는 번호들
    candidates = list(top_numbers_set - last_numbers_set)
    
    predictions = []
    for _ in range(num_predictions):
        # 마지막 당첨 번호에서 3개를 유지하고, 상위 번호에서 3개를 선택
        new_numbers = random.sample(last_numbers, 3) + random.sample(candidates, 3)
        predictions.append(sorted(new_numbers))
    
    return predictions

file_path = 'winner_number.xlsx'
winning_numbers = load_winning_numbers(file_path)

print(f"총 {len(winning_numbers)}개의 당첨 번호를 로드했습니다.")

transitions = calculate_transition_probabilities(winning_numbers)
last_winning_numbers = winning_numbers[-1]

print("\n마지막 당첨 번호:", last_winning_numbers)

predictions = predict_next_numbers(last_winning_numbers, transitions)

print("\n전이 확률 기반 다음 회차 예측 (상위 5개):")
for i, (pred, prob) in enumerate(predictions, 1):
    relative_prob = prob / (1/8145060)
    print(f"{i}. 번호: {pred}")
    print(f"   상대적 확률: 무작위 선택 대비 {relative_prob:.2f}배")
    print(f"   절대 확률: {prob:.10f} ({prob*100:.8f}%)")

top_numbers, match_distribution, average_matches = analyze_top_numbers_winning(winning_numbers)

print("\n가장 자주 나온 번호 (상위 10개):")
for num, freq in top_numbers:
    print(f"번호 {num}: {freq:.2%}")

expected_value = calculate_expected_value(dict(top_numbers))
print(f"\n상위 10개 번호의 당첨 기대값: {expected_value:.4f}")

top_number_predictions = predict_with_top_numbers(last_winning_numbers, top_numbers)
print("\n상위 번호를 고려한 다음 회차 예측 (상위 5개):")
for i, pred in enumerate(top_number_predictions, 1):
    print(f"{i}. 번호: {pred}")

# 두 예측 방법의 결과 비교
print("\n두 예측 방법 비교:")
for i in range(5):
    transition_pred = set(predictions[i][0])
    top_number_pred = set(top_number_predictions[i])
    common_numbers = transition_pred & top_number_pred
    print(f"예측 {i+1}: 공통 번호 {len(common_numbers)}개 - {sorted(common_numbers)}")

총 1139개의 당첨 번호를 로드했습니다.

마지막 당첨 번호: [5, 12, 15, 30, 37, 40]

전이 확률 기반 다음 회차 예측 (상위 5개):
1. 번호: [6, 15, 16, 19, 30, 45]
   상대적 확률: 무작위 선택 대비 1.00배
   절대 확률: 0.0000001228 (0.00001228%)
2. 번호: [24, 26, 33, 36, 44, 45]
   상대적 확률: 무작위 선택 대비 1.00배
   절대 확률: 0.0000001228 (0.00001228%)
3. 번호: [1, 4, 11, 12, 27, 32]
   상대적 확률: 무작위 선택 대비 1.00배
   절대 확률: 0.0000001228 (0.00001228%)
4. 번호: [4, 6, 12, 17, 29, 30]
   상대적 확률: 무작위 선택 대비 1.00배
   절대 확률: 0.0000001228 (0.00001228%)
5. 번호: [1, 16, 19, 34, 36, 41]
   상대적 확률: 무작위 선택 대비 1.00배
   절대 확률: 0.0000001228 (0.00001228%)

상위 10개 번호 중 당첨된 번호 개수 분포:
0개 일치: 182회 (15.98%)
1개 일치: 457회 (40.12%)
2개 일치: 340회 (29.85%)
3개 일치: 131회 (11.50%)
4개 일치: 29회 (2.55%)

평균 일치 개수: 1.45

가장 자주 나온 번호 (상위 10개):
번호 34: 15.45%
번호 12: 14.57%
번호 14: 14.49%
번호 13: 14.49%
번호 18: 14.49%
번호 33: 14.40%
번호 45: 14.40%
번호 40: 14.22%
번호 27: 14.05%
번호 17: 13.96%

상위 10개 번호의 당첨 기대값: 0.8671

상위 번호를 고려한 다음 회차 예측 (상위 5개):
1. 번호: [12, 13, 17, 27, 37, 40]
2. 번호: [5, 12, 13, 17, 33, 40]
3. 번호: [5